<a href="https://colab.research.google.com/github/pphos/pytorch_tutorial/blob/master/autograd_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Autograd: Automatic Differentiaion

PyTorchのすべてのニューラルネットワークの中心にあるものがautogradパッケージです.
最初にこれを簡単にみて, ニューラルネットワークの訓練をしてみましょう！

autogradパッケージは, テンソル上のすべての演算に対して自動微分を提供します.
autogradはDefine-by-runフレームワークで, コードの実行方法によって逆伝搬が定義され, 各イテレーションごとに異なるものを使用できます.

いくつかの例を挙げて, これをより簡単な言葉で見ていきましょう

## Tensor
`torch.Tensor`はパッケージの中心クラスです.
`.require_grad`属性を`True`に設定すると, それに対するすべての操作を追跡し始めます.
計算が終了したら`.backward()`を呼び出して, すべての勾配を自動的に計算させることができます.
このテンソルの勾配は`.grad`属性に蓄積されます.

テンソルが履歴を追跡しないようにするには, `detach()`を呼び出して計算履歴からテンソルを切り離し, 将来の計算が追跡されないようにします.

履歴を追跡しないようにするためには, コードブロックを`torch.no_grad():`で囲むことでもできます.
モデルが`requires_grad=True`で学習可能なパラメータを持っていても,
勾配が必要ない場合があるため, モデルを評価する際に便利です.

`autograd`の実装には`Function`というもう一つの重要なクラスがあります.
`Tensor`と`Function`は相互に接続されており, 計算の完全な履歴をエンコードする非周期グラフを構築します.

各テンソルは, テンソルを作成した関数を参照する`.grad_fn`属性を持っています.
微分を計算したい場合は, テンソルに対して`.backward()`を呼び出すことができます.

テンソルがスカラ(つまり1つの要素のデータを保持している)であれば, backward()に引数を指定する必要はありませんが, 要素数が多い場合は, 形状が一致するテンソルである勾配の引数を指定する必要があります.



In [0]:
import torch

テンソルを作成し, `requires_grad=True`を設定して, テンソルで計算を追跡します.

In [12]:
x = torch.ones(2, 2, requires_grad=True)
print(x)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


テンソルの操作を行います:

In [13]:
y = x + 2
print(y)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


`y`は操作の結果として作成されたので, `grad_fn` があります.

In [14]:
print(y.grad_fn)

`y`でされに操作を行います

In [15]:
z = y * y * 3
out = z.mean()

print(z, out)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>) tensor(27., grad_fn=<MeanBackward0>)


`.requires_grad_(...)`は, 既存のTensorの`requires_grad`フラグを代わりに変更します.
入力フラグが与えられていない場合のデフォルトは`False`です.

In [16]:
a = torch.randn(2, 2)
a = ((a * 3) / (a - 1))
print(a.requires_grad)
a.requires_grad_(True)
print(a.requires_grad)
b = (a * a).sum()
print(b.grad_fn)

False
True


# Gradients
ここで逆伝搬をしてみましょう.
`out`はスカラを1つ含むので, `out.backward()`は`out.backward(torch.tensor(1))`と等価です.

In [0]:
out.backward()

勾配 d(out)/dxを表示しましょう

In [19]:
print(x.grad)

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


各要素が4.5である行列が表示されるはずです.
このテンソルを"$o$"と呼びましょう.

現在$o$は以下の値を持っています.

$o = \frac{1}{4}\sum_i z_i$, $z_i = 3(x_i +2)^2$ $z_i\bigr\rvert_{x_i=1} = 27$.

それゆえに$o$の各要素の微分は以下のようになります.

$\frac{\partial o}{\partial x_i} = \frac{3}{2}(x_i + 2)$

$\frac{\partial o}{\partial x_i}\bigr\rvert_{x_i=1} = \frac{9}{2} = 4.5$


autogradではクレイジーなことがたくさんできます.

In [21]:
x = torch.randn(3, requires_grad=True)

y = x * 2
while y.data.norm() < 1000:
  y = y * 2

print(y)

tensor([  -73.8176, -1202.9667,   136.7958], grad_fn=<MulBackward0>)


In [22]:
gradients = torch.tensor([0.1, 1.0, 0.0001], dtype=torch.float)
y.backward(gradients)

print(x.grad)

tensor([5.1200e+01, 5.1200e+02, 5.1200e-02])


`requires_grad=True`の場合は, コードブロックを`torch.no_grad()`でラップすることで, テンソル上でautogradが履歴を追跡しないようにすることもできます.

In [23]:
print(x.requires_grad)
print((x ** 2).requires_grad)

with torch.no_grad():
  print((x ** 2).requires_grad)

True
True
False
